#  Прогнозирование заказов такси

Компания «Чётенькое такси» собрала исторические данные о заказах такси в аэропортах. 

**Цель исследования** - тобы привлекать больше водителей в период пиковой нагрузки, нужно спрогнозировать количество заказов такси на следующий час. Построить модель для такого предсказания. Значение метрики *RMSE* на тестовой выборке должно быть не больше 48.

Мне нужно:

1. Загрузить данные и выполнить их ресемплирование по одному часу.
2. Проанализировать данные.
3. Обучить разные модели с различными гиперпараметрами. Сделать тестовую выборку размером 10% от исходных данных.
4. Проверить данные на тестовой выборке и сделать выводы.

Данные лежат в файле `taxi.csv`. Количество заказов находится в столбце `num_orders` (от англ. *number of orders*, «число заказов»).

## Подготовка

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

from statsmodels.tsa.seasonal import seasonal_decompose
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split

from statsmodels.tsa.stattools import adfuller

from catboost import CatBoostRegressor, cv, Pool

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import ParameterGrid
from tqdm.notebook import tqdm
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import TimeSeriesSplit

In [28]:
df = pd.read_csv('datasets/taxi.csv', index_col=[0], parse_dates = [0])

In [29]:
df

,num_orders
datetime,
2018-03-01 00:00:00,9
2018-03-01 00:10:00,14
2018-03-01 00:20:00,28
2018-03-01 00:30:00,20
2018-03-01 00:40:00,32
...,...
2018-08-31 23:10:00,32
2018-08-31 23:20:00,24
2018-08-31 23:30:00,27


In [30]:
fig = px.line(df, x = df.index , y = 'num_orders')
fig.show()

Сделаем ресемплинг по 1 дню

In [31]:
df = df.resample('1H').sum()

In [32]:
fig = px.line(df, x = df.index , y = 'num_orders')
fig.show()

## Анализ

### Подготовка данных

Cделаем декомпозицию ряда для выделения тренда, сезонности и остатков

In [33]:
decomposed = seasonal_decompose(df)

In [34]:
seasonal = decomposed.seasonal

In [35]:
trend = decomposed.trend

In [36]:
resid = decomposed.resid

In [37]:
fig = px.line(x=seasonal.index, y=seasonal, title='Seasonal', range_x=['2018-03-01','2018-03-5'])
fig.show()

Период составляет 1 день.

In [38]:
fig = make_subplots(rows=3, cols=1, subplot_titles=("Seasonal","Trend", "Resid"))

fig.append_trace(go.Scatter(
    x=seasonal.index,
    y=seasonal,
), row=1, col=1)

fig.append_trace(go.Scatter(
    x=trend.index,
    y=trend,
), row=2, col=1)

fig.append_trace(go.Scatter(
    x=resid.index,
    y=resid,
), row=3, col=1)

fig.update_layout(height=800, width=1_600, title_text="Time series decomposition")
fig.show()

Тест Дики - Фуллера на станционраность

In [39]:
test_adfuller = adfuller(df)

In [40]:
print('adf: ', test_adfuller[0])
print('p-value: ', test_adfuller[1])
print('Critical values: ', test_adfuller[4])
if test_adfuller[0] > test_adfuller[4]['5%']:
    print('есть единичные корни, ряд не станционарен')
else:
    print('единичных корней нет, ряд станционарен')

adf:  -3.068924289027957
p-value:  0.0289400514026128
Critical values:  {'1%': -3.431842162413052, '5%': -2.8621993540813637, '10%': -2.567120978470452}
единичных корней нет, ряд станционарен


### Создание новых фичей для обучения

In [41]:
df

,num_orders
datetime,
2018-03-01 00:00:00,124
2018-03-01 01:00:00,85
2018-03-01 02:00:00,71
2018-03-01 03:00:00,66
2018-03-01 04:00:00,43
...,...
2018-08-31 19:00:00,136
2018-08-31 20:00:00,154
2018-08-31 21:00:00,159


In [42]:
def make_features(data, max_lag, rolling_mean_size):
    data['year'] = data.index.year
    data['month'] = data.index.month
    data['day'] = data.index.day
    data['dayofweek'] = data.index.dayofweek
    data['hour'] = data.index.hour
    
    for lag in range(1, max_lag + 1):
        data['lag_{}'.format(lag)] = data['num_orders'].shift(lag)

    data['rolling_mean'] = data['num_orders'].shift().rolling(rolling_mean_size).mean()
    
    return data

In [43]:
df_model = make_features(df,40,40)

In [44]:
df_model

,num_orders,year,month,day,dayofweek,hour,lag_1,lag_2,lag_3,lag_4,...,lag_32,lag_33,lag_34,lag_35,lag_36,lag_37,lag_38,lag_39,lag_40,rolling_mean
datetime,,,,,,,,,,,,,,,,,,,,,
2018-03-01 00:00:00,124,2018,3,1,3,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-03-01 01:00:00,85,2018,3,1,3,1,124.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-03-01 02:00:00,71,2018,3,1,3,2,85.0,124.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-03-01 03:00:00,66,2018,3,1,3,3,71.0,85.0,124.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-03-01 04:00:00,43,2018,3,1,3,4,66.0,71.0,85.0,124.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-08-31 19:00:00,136,2018,8,31,4,19,207.0,217.0,197.0,116.0,...,156.0,227.0,226.0,166.0,39.0,13.0,56.0,186.0,193.0,164.150
2018-08-31 20:00:00,154,2018,8,31,4,20,136.0,207.0,217.0,197.0,...,173.0,156.0,227.0,226.0,166.0,39.0,13.0,56.0,186.0,162.725
2018-08-31 21:00:00,159,2018,8,31,4,21,154.0,136.0,207.0,217.0,...,154.0,173.0,156.0,227.0,226.0,166.0,39.0,13.0,56.0,161.925


In [45]:
train, test = train_test_split(df_model, shuffle=False, test_size=0.1)

In [46]:
train = train.dropna()

In [47]:
train

,num_orders,year,month,day,dayofweek,hour,lag_1,lag_2,lag_3,lag_4,...,lag_32,lag_33,lag_34,lag_35,lag_36,lag_37,lag_38,lag_39,lag_40,rolling_mean
datetime,,,,,,,,,,,,,,,,,,,,,
2018-03-02 16:00:00,82,2018,3,2,4,16,59.0,44.0,45.0,36.0,...,34.0,15.0,12.0,6.0,43.0,66.0,71.0,85.0,124.0,54.550
2018-03-02 17:00:00,83,2018,3,2,4,17,82.0,59.0,44.0,45.0,...,69.0,34.0,15.0,12.0,6.0,43.0,66.0,71.0,85.0,53.500
2018-03-02 18:00:00,89,2018,3,2,4,18,83.0,82.0,59.0,44.0,...,64.0,69.0,34.0,15.0,12.0,6.0,43.0,66.0,71.0,53.450
2018-03-02 19:00:00,49,2018,3,2,4,19,89.0,83.0,82.0,59.0,...,96.0,64.0,69.0,34.0,15.0,12.0,6.0,43.0,66.0,53.900
2018-03-02 20:00:00,93,2018,3,2,4,20,49.0,89.0,83.0,82.0,...,30.0,96.0,64.0,69.0,34.0,15.0,12.0,6.0,43.0,53.475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-08-13 09:00:00,137,2018,8,13,0,9,91.0,39.0,66.0,83.0,...,177.0,174.0,155.0,89.0,105.0,84.0,102.0,62.0,122.0,122.125
2018-08-13 10:00:00,156,2018,8,13,0,10,137.0,91.0,39.0,66.0,...,157.0,177.0,174.0,155.0,89.0,105.0,84.0,102.0,62.0,122.500
2018-08-13 11:00:00,144,2018,8,13,0,11,156.0,137.0,91.0,39.0,...,136.0,157.0,177.0,174.0,155.0,89.0,105.0,84.0,102.0,124.850


In [48]:
X = train.drop(['num_orders','year'], axis = 1)
y = train['num_orders']

In [49]:
X

,month,day,dayofweek,hour,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,...,lag_32,lag_33,lag_34,lag_35,lag_36,lag_37,lag_38,lag_39,lag_40,rolling_mean
datetime,,,,,,,,,,,,,,,,,,,,,
2018-03-02 16:00:00,3,2,4,16,59.0,44.0,45.0,36.0,91.0,54.0,...,34.0,15.0,12.0,6.0,43.0,66.0,71.0,85.0,124.0,54.550
2018-03-02 17:00:00,3,2,4,17,82.0,59.0,44.0,45.0,36.0,91.0,...,69.0,34.0,15.0,12.0,6.0,43.0,66.0,71.0,85.0,53.500
2018-03-02 18:00:00,3,2,4,18,83.0,82.0,59.0,44.0,45.0,36.0,...,64.0,69.0,34.0,15.0,12.0,6.0,43.0,66.0,71.0,53.450
2018-03-02 19:00:00,3,2,4,19,89.0,83.0,82.0,59.0,44.0,45.0,...,96.0,64.0,69.0,34.0,15.0,12.0,6.0,43.0,66.0,53.900
2018-03-02 20:00:00,3,2,4,20,49.0,89.0,83.0,82.0,59.0,44.0,...,30.0,96.0,64.0,69.0,34.0,15.0,12.0,6.0,43.0,53.475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-08-13 09:00:00,8,13,0,9,91.0,39.0,66.0,83.0,143.0,168.0,...,177.0,174.0,155.0,89.0,105.0,84.0,102.0,62.0,122.0,122.125
2018-08-13 10:00:00,8,13,0,10,137.0,91.0,39.0,66.0,83.0,143.0,...,157.0,177.0,174.0,155.0,89.0,105.0,84.0,102.0,62.0,122.500
2018-08-13 11:00:00,8,13,0,11,156.0,137.0,91.0,39.0,66.0,83.0,...,136.0,157.0,177.0,174.0,155.0,89.0,105.0,84.0,102.0,124.850


### Вывод

- провели декомпозицию ряда к 1 дню
- по результатам теста Дики - Фулера ряд станционарный
- добавил новые фичи - 40 лагов и 1 скользящую среднюю

## Обучение

### CatboostRegressor

In [50]:
params = {
    'loss_function': 'RMSE',
    'iterations': 5_000,
    'random_seed': 42,
    'learning_rate': 0.001,
    'early_stopping_rounds': 100
}

In [51]:
cv_data = cv(
    params=params,
    pool=Pool(X, label=y),
    fold_count=5, # Разбивка выборки на 5 кусочков
    partition_random_seed=42,
    plot=True, # Никуда без визуализатора
    stratified=True, 
    verbose=False,
    shuffle=False,
    type = 'TimeSeries'
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/5]

bestTest = 22.4907704
bestIteration = 4999

Training on fold [1/5]

bestTest = 23.92702473
bestIteration = 4999

Training on fold [2/5]

bestTest = 24.673205
bestIteration = 4999

Training on fold [3/5]

bestTest = 26.13564305
bestIteration = 4999

Training on fold [4/5]

bestTest = 35.71829893
bestIteration = 4999



In [52]:
cv_data

,iterations,test-RMSE-mean,test-RMSE-std,train-RMSE-mean,train-RMSE-std
0,0,89.944727,18.112790,71.695825,6.487534
1,1,89.886025,18.102658,71.638320,6.478839
2,2,89.831128,18.092023,71.581159,6.471436
3,3,89.767319,18.082240,71.522645,6.465541
4,4,89.703813,18.069898,71.463528,6.458272
...,...,...,...,...,...
4995,4995,26.591728,5.271467,17.420595,2.170371
4996,4996,26.591033,5.271277,17.419512,2.170784
4997,4997,26.590167,5.270484,17.418679,2.170866
4998,4998,26.589701,5.270480,17.417868,2.171122


Имеем лучшее значение RMSE в результате проверки модели кросс-валидацией 26.588988

### RandomForestRegresssor

In [53]:
X

,month,day,dayofweek,hour,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,...,lag_32,lag_33,lag_34,lag_35,lag_36,lag_37,lag_38,lag_39,lag_40,rolling_mean
datetime,,,,,,,,,,,,,,,,,,,,,
2018-03-02 16:00:00,3,2,4,16,59.0,44.0,45.0,36.0,91.0,54.0,...,34.0,15.0,12.0,6.0,43.0,66.0,71.0,85.0,124.0,54.550
2018-03-02 17:00:00,3,2,4,17,82.0,59.0,44.0,45.0,36.0,91.0,...,69.0,34.0,15.0,12.0,6.0,43.0,66.0,71.0,85.0,53.500
2018-03-02 18:00:00,3,2,4,18,83.0,82.0,59.0,44.0,45.0,36.0,...,64.0,69.0,34.0,15.0,12.0,6.0,43.0,66.0,71.0,53.450
2018-03-02 19:00:00,3,2,4,19,89.0,83.0,82.0,59.0,44.0,45.0,...,96.0,64.0,69.0,34.0,15.0,12.0,6.0,43.0,66.0,53.900
2018-03-02 20:00:00,3,2,4,20,49.0,89.0,83.0,82.0,59.0,44.0,...,30.0,96.0,64.0,69.0,34.0,15.0,12.0,6.0,43.0,53.475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-08-13 09:00:00,8,13,0,9,91.0,39.0,66.0,83.0,143.0,168.0,...,177.0,174.0,155.0,89.0,105.0,84.0,102.0,62.0,122.0,122.125
2018-08-13 10:00:00,8,13,0,10,137.0,91.0,39.0,66.0,83.0,143.0,...,157.0,177.0,174.0,155.0,89.0,105.0,84.0,102.0,62.0,122.500
2018-08-13 11:00:00,8,13,0,11,156.0,137.0,91.0,39.0,66.0,83.0,...,136.0,157.0,177.0,174.0,155.0,89.0,105.0,84.0,102.0,124.850


In [54]:
tscv = TimeSeriesSplit(n_splits=5)
print(tscv)

TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None)


In [55]:
for fold, (train_index, test_index) in enumerate(tscv.split(X)):
    print("Fold: {}".format(fold))
    print("TRAIN indices:", train_index, "\n", "TEST indices:", test_index)
    print("\n")
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index,:]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

Fold: 0
TRAIN indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240

In [56]:
best_model = None
best_result = 50

In [57]:
param_grid = {'max_depth': list(range(15, 18)), 'n_estimators': list(range(120,140))}
params = list(ParameterGrid(param_grid))
for param in tqdm(params):
    model = RandomForestRegressor(random_state=42, max_depth=param['max_depth'], n_estimators=param['n_estimators'], n_jobs= -1)
    scores = []
    for fold, (train_index, test_index) in enumerate(tscv.split(X)):
        X_train, X_test = X.iloc[train_index, :], X.iloc[test_index,:]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        model.fit(X_train,y_train)
        pred = model.predict(X_test)
        score = mean_squared_error(y_test,pred)**0.5
        scores.append(score)
    final_score = (sum(scores) / len(scores))
    if best_result > final_score:
        best_result = final_score
        best_model = model

  0%|          | 0/60 [00:00<?, ?it/s]

In [58]:
print('Best model is:',best_model)
print('Best RMSE metric is:',best_result)

Best model is: RandomForestRegressor(max_depth=16, n_estimators=130, n_jobs=-1,
                      random_state=42)
Best RMSE metric is: 25.27486005815091


### Вывод

- Лучший результат на катбусте 26.588988
- Лучший результат на случайном лесе 25.378890690107987

## Тестирование

In [59]:
X_test = test.drop(['num_orders','year'], axis = 1)
y_test = test['num_orders']

Обучаю лучшие модели на трейне

### RandomForestRegresssor

In [60]:
print('Best model is:',best_model)
print('Best RMSE metric is:',best_result)

Best model is: RandomForestRegressor(max_depth=16, n_estimators=130, n_jobs=-1,
                      random_state=42)
Best RMSE metric is: 25.27486005815091


In [61]:
best_model.fit(X,y)

RandomForestRegressor(max_depth=16, n_estimators=130, n_jobs=-1,
                      random_state=42)

In [62]:
pred = best_model.predict(X_test)

In [63]:
rf_rmse = mean_squared_error(y_test,pred)**0.5

In [64]:
rf_rmse

43.543878315575455

In [65]:
# print('Лучший результат на CatBoostRegressor: ', cbr_rmse)
print('Лучший результат на RandomForestRegressor: ', rf_rmse)

Лучший результат на RandomForestRegressor:  43.543878315575455


In [66]:
fig = go.Figure([
    go.Scatter(
        name='True',
        x=test.index,
        y=test['num_orders'],
        mode='lines',
        marker=dict(color='red', size=2),
        showlegend=True
    ),
    go.Scatter(
        name='Pred',
        x=test.index,
        y=pred,
        mode='lines',
        marker=dict(color="#444", size=2),
        showlegend=True
    )
])

fig.update_layout(height=600, width=1_600, title_text="True and predicted values")

fig.show()